In [1]:
from datetime import datetime
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-04-01 17:41:35.203748


In [0]:
import torch
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def get_train_loader():

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                              shuffle=True, num_workers=0)
    return train_loader

def get_test_loader():
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=32,
                                             shuffle=False, num_workers=0)
    return test_loader

def get_classes():
    classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    return classes


In [0]:
import sys
sys.path.append('/content/drive/My Drive')


In [4]:

from quiz9_dnn import Net
from torchsummary import summary
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
              ReLU-2           [-1, 64, 32, 32]               0
       BatchNorm2d-3           [-1, 64, 32, 32]             128
           Dropout-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,864
              ReLU-6           [-1, 64, 32, 32]               0
       BatchNorm2d-7           [-1, 64, 32, 32]             128
           Dropout-8           [-1, 64, 32, 32]               0
            Conv2d-9           [-1, 64, 32, 32]          36,864
             ReLU-10           [-1, 64, 32, 32]               0
      BatchNorm2d-11           [-1, 64, 32, 32]             128
        MaxPool2d-12           [-1, 64, 16, 16]               0
           Conv2d-13           [-1, 64, 16, 16]          36,864
             ReLU-14           [-1

In [0]:
from __future__ import print_function
import torch.nn as nn
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch,train_losses,train_acc):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        y_pred = model(data)

        # Calculate loss
        criteria = nn.CrossEntropyLoss()
        loss = criteria(y_pred, target)
        train_losses.append(loss)

        # Backpropagation
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100 * correct / processed:0.2f}')
        train_acc.append(100 * correct / processed)
    return train_acc




In [0]:
from __future__ import print_function
import torch
import torch.nn.functional as F


def test(model, device, test_loader,test_losses,test_acc):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_acc

In [7]:
from __future__ import print_function

import torch
from torchsummary import summary
import torch.optim as optim

# from models.resnet18 import ResNet18
# from data_loader.data_loader_cifar import get_train_loader
# from data_loader.data_loader_cifar import get_test_loader
# from training.training import train
# from scoring.scoring import test


# Set seed for all the environments
SEED = 1
torch.manual_seed(SEED)

CUDA = torch.cuda.is_available()
print("CUDA is available:",CUDA)
# If CUDA is available the set SEED for it
if CUDA:
    torch.cuda.manual_seed(SEED)

from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

train_loader = get_train_loader()
test_loader = get_test_loader()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 10
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch,train_losses,train_acc)
    test(model, device, test_loader,test_losses,test_acc)

CUDA is available: True
Files already downloaded and verified
Files already downloaded and verified


Loss=2.249957799911499 Batch_id=3 Accuracy=18.75:   0%|          | 3/1563 [00:00<01:10, 21.98it/s]

EPOCH: 0


Loss=1.6394864320755005 Batch_id=1562 Accuracy=54.46: 100%|██████████| 1563/1563 [01:05<00:00, 23.74it/s]
Loss=0.972696840763092 Batch_id=3 Accuracy=64.84:   0%|          | 3/1563 [00:00<01:12, 21.48it/s] 


Test set: Average loss: -4.4516, Accuracy: 5842/10000 (58.42%)

EPOCH: 1


...


Test set: Average loss: -8.3046, Accuracy: 7947/10000 (79.47%)

EPOCH: 8


Loss=0.3895673453807831 Batch_id=1562 Accuracy=86.13: 100%|██████████| 1563/1563 [01:05<00:00, 23.99it/s]
Loss=0.2841266989707947 Batch_id=3 Accuracy=90.62:   0%|          | 3/1563 [00:00<01:08, 22.63it/s] 


Test set: Average loss: -8.1176, Accuracy: 7961/10000 (79.61%)

EPOCH: 9


Loss=0.5241731405258179 Batch_id=1562 Accuracy=87.12: 100%|██████████| 1563/1563 [01:05<00:00, 23.97it/s]



Test set: Average loss: -8.3961, Accuracy: 7950/10000 (79.50%)



In [8]:
  3584/224


16.0

In [9]:
Net()

Net(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, inplace=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, inplace=False)
  )
  (conv_block_3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_block_4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): BatchNorm2d